In [ ]:
import numpy as np
import plotly.graph_objects as go

# ---------- Proper replicator dynamics for RPS ----------
def payoff_matrix(rho: float) -> np.ndarray:
    # A_ij = payoff to i when facing j
    return np.array([
        [0.0, -rho,  1.0],   # Rock vs (R,P,S)
        [1.0,  0.0, -rho],   # Paper
        [-rho, 1.0,  0.0]    # Scissors
    ], dtype=float)

def replicator_rhs(x: np.ndarray, rho: float) -> np.ndarray:
    A = payoff_matrix(rho)
    f = A @ x
    fbar = float(x @ f)
    return x * (f - fbar)  # elementwise

def rk4_path(x0, rho, dt=0.02, steps=3500):
    x = np.array(x0, dtype=float)
    x = np.clip(x, 1e-12, 1.0)
    x /= x.sum()
    traj = np.zeros((steps+1, 3))
    traj[0] = x
    for k in range(steps):
        k1 = replicator_rhs(x, rho)
        k2 = replicator_rhs(x + 0.5*dt*k1, rho)
        k3 = replicator_rhs(x + 0.5*dt*k2, rho)
        k4 = replicator_rhs(x + dt*k3, rho)
        x = x + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)

        # keep on the simplex
        x = np.maximum(x, 0.0)
        s = x.sum()
        x = (x / s) if s > 0 else np.array([1/3, 1/3, 1/3], dtype=float)
        traj[k+1] = x
    return traj

# ---------- Same initial point for all ----------
x0 = [0.5, 0.3, 0.2]

# Three regimes
traj_in  = rk4_path(x0, rho=0.8, steps=3500, dt=0.02)  # spirals inward
traj_eq  = rk4_path(x0, rho=1.0, steps=3000, dt=0.02)  # neutral cycle
traj_out = rk4_path(x0, rho=1.2, steps=3500, dt=0.02)  # spirals outward

# ---------- Simplex edges ----------
corners = np.array([
    [1.0, 0.0, 0.0],  # R
    [0.0, 1.0, 0.0],  # P
    [0.0, 0.0, 1.0],  # S
    [1.0, 0.0, 0.0],  # close
])
edge_trace = go.Scatter3d(
    x=corners[:,0], y=corners[:,1], z=corners[:,2],
    mode='lines',
    line=dict(width=4),
    name='Simplex edges',
    hoverinfo='skip',
    showlegend=False
)

# ---------- Fixed points: three corners + interior ----------
fixed_pts = np.array([
    [1.0, 0.0, 0.0],   # R
    [0.0, 1.0, 0.0],   # P
    [0.0, 0.0, 1.0],   # S
    [1/3, 1/3, 1/3],   # interior coexistence
], dtype=float)
fixed_names = ['R', 'P', 'S', '(1/3,1/3,1/3)']

fixed_trace = go.Scatter3d(
    x=fixed_pts[:,0], y=fixed_pts[:,1], z=fixed_pts[:,2],
    mode='markers+text',
    text=fixed_names,
    textposition='top center',
    marker=dict(size=6),
    name='Fixed points'
)

# ---------- Build figure ----------
fig = go.Figure()

# simplex edges
fig.add_trace(edge_trace)

# trajectories (solid, different colors; same start point)
fig.add_trace(go.Scatter3d(
    x=traj_in[:,0], y=traj_in[:,1], z=traj_in[:,2],
    mode='lines', name='ρ = 0.8 (inward)',
    line=dict(width=6)
))
fig.add_trace(go.Scatter3d(
    x=traj_eq[:,0], y=traj_eq[:,1], z=traj_eq[:,2],
    mode='lines', name='ρ = 1.0 (neutral)',
    line=dict(width=6)
))
fig.add_trace(go.Scatter3d(
    x=traj_out[:,0], y=traj_out[:,1], z=traj_out[:,2],
    mode='lines', name='ρ = 1.2 (outward)',
    line=dict(width=6)
))

# fixed points
fig.add_trace(fixed_trace)

# layout: 1 vertical to 1.2 horizontal (set x=y=1.2, z=1)
fig.update_layout(
    title='Replicator Dynamics on the Simplex (R, P, S)',
    scene=dict(
        xaxis_title='R',
        yaxis_title='P',
        zaxis_title='S',
        xaxis=dict(range=[0,1]),
        yaxis=dict(range=[0,1]),
        zaxis=dict(range=[0,1]),
        aspectmode='manual',
        aspectratio=dict(x=1.2, y=1.2, z=1.0),
        camera=dict(eye=dict(x=1.6, y=1.6, z=1.2))
    ),
    legend=dict(itemsizing='constant')
)

fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# ---------- Proper replicator dynamics for RPS ----------
def payoff_matrix(rho: float) -> np.ndarray:
    # A_ij = payoff to i when facing j
    return np.array([
        [0.0, -rho,  1.0],   # Rock vs (R,P,S)
        [1.0,  0.0, -rho],   # Paper
        [-rho, 1.0,  0.0]    # Scissors
    ], dtype=float)

def replicator_rhs(x: np.ndarray, rho: float) -> np.ndarray:
    A = payoff_matrix(rho)
    f = A @ x
    fbar = float(x @ f)
    return x * (f - fbar)  # elementwise

def rk4_path(x0, rho, dt=0.02, steps=3500):
    x = np.array(x0, dtype=float)
    x = np.clip(x, 1e-12, 1.0)
    x /= x.sum()
    traj = np.zeros((steps+1, 3))
    traj[0] = x
    for k in range(steps):
        k1 = replicator_rhs(x, rho)
        k2 = replicator_rhs(x + 0.5*dt*k1, rho)
        k3 = replicator_rhs(x + 0.5*dt*k2, rho)
        k4 = replicator_rhs(x + dt*k3, rho)
        x = x + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)

        # keep on the simplex
        x = np.maximum(x, 0.0)
        s = x.sum()
        x = (x / s) if s > 0 else np.array([1/3, 1/3, 1/3], dtype=float)
        traj[k+1] = x
    return traj

# ---------- Parameters ----------
rho = 1.0
x0 = [0.5, 0.3, 0.2]
traj = rk4_path(x0, rho=rho, steps=3500, dt=0.02)

# ---------- Simplex edges ----------
corners = np.array([
    [1.0, 0.0, 0.0],  # R
    [0.0, 1.0, 0.0],  # P
    [0.0, 0.0, 1.0],  # S
    [1.0, 0.0, 0.0],  # close
])
edge_trace = go.Scatter3d(
    x=corners[:,0], y=corners[:,1], z=corners[:,2],
    mode='lines',
    line=dict(width=4, color='black'),
    name='Simplex edges',
    hoverinfo='skip',
    showlegend=False
)

# ---------- Trajectory (ρ = 1) ----------
traj_trace = go.Scatter3d(
    x=traj[:,0], y=traj[:,1], z=traj[:,2],
    mode='lines',
    name='ρ = 1.0 (neutral)',
    line=dict(width=6, color='royalblue')
)

# ---------- Build figure ----------
fig = go.Figure()

# simplex edges and trajectory
fig.add_trace(edge_trace)
fig.add_trace(traj_trace)

# ---------- Layout ----------
fig.update_layout(
    title='Replicator Dynamics on the Simplex (R, P, S)',
    scene=dict(
        xaxis_title='R',
        yaxis_title='P',
        zaxis_title='S',
        xaxis=dict(range=[0,1]),
        yaxis=dict(range=[0,1]),
        zaxis=dict(range=[0,1]),
        aspectmode='manual',
        aspectratio=dict(x=1.2, y=1.2, z=1.0),
        camera=dict(eye=dict(x=1.6, y=1.6, z=1.2))
    ),
    legend=dict(itemsizing='constant'),
    margin=dict(l=0, r=0, t=50, b=0)
)

fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# ρ range (slightly extended to capture full curve)
rho = np.linspace(-3, 5, 600)

# Trace and determinant of the reduced Jacobian
trace = (rho - 1) / 3
det = (rho**2 + rho + 1) / 9

# Create figure
fig = go.Figure()

# Parabola curve
fig.add_trace(go.Scatter(
    x=trace, y=det,
    mode='lines',
    line=dict(color='black', width=3),
    name='Replicator Jacobian locus'
))

# Shaded regions for stability
fig.add_vrect(
    x0=-1.4, x1=0, fillcolor='lightblue', opacity=0.3,
    annotation_text='Stable spiral (ρ < 1)', annotation_position='bottom left',
    layer='below', line_width=0
)
fig.add_vrect(
    x0=0, x1=1, fillcolor='salmon', opacity=0.3,
    annotation_text='Unstable spiral (ρ > 1)', annotation_position='bottom right',
    layer='below', line_width=0
)

# Neutral point at ρ = 1
tr_1 = (1 - 1) / 3
det_1 = (1**2 + 1 + 1) / 9
fig.add_trace(go.Scatter(
    x=[tr_1], y=[det_1],
    mode='markers+text',
    text=['ρ = 1'],
    textposition='bottom right',
    marker=dict(size=8, color='black'),
    name='Neutral (ρ = 1)'
))

# Layout and axes formatting
fig.update_layout(
    title='Trace–Determinant Diagram of the Replicator System',
    xaxis_title='Trace(J)',
    yaxis_title='Determinant(J)',
    template='simple_white',
    width=750, height=520,
    font=dict(size=13),
    showlegend=False
)

# Adjusted viewing window (zoomed to -1.4 → 1)
fig.update_xaxes(range=[-1.4, 1], zeroline=True, zerolinewidth=1, zerolinecolor='gray')
fig.update_yaxes(range=[0, 0.6], zeroline=True, zerolinewidth=1, zerolinecolor='gray')

# Optional static save
# fig.write_image("trace_determinant_diagram_zoomed.png", scale=3)

fig.show()
